# Motivating Example: 
As the pizza shop owner, your developer gives you two versions of program to manage your daily workflow.

Which version do you prefer?

### Project structure of version 1

```bash
main.py      # main file
```

In [5]:
# version 1

from datetime import datetime

MENU = {
    "pizza": {
        "margherita": {"sizes": {"S": 8.50, "M": 11.00, "L": 13.50}},
        "pepperoni":  {"sizes": {"S": 9.50, "M": 12.50, "L": 15.00}},
    },
    "drink": {
        "soda": {"sizes": {"S": 2.00, "M": 2.50, "L": 3.00}},
        "tea":  {"sizes": {"S": 2.00, "M": 2.50, "L": 3.00}},
    },
    "dessert": {"tiramisu": {"price": 5.00}},
}

FOOD_TAX = 0.08
DRINK_TAX = 0.10
DELIVERY_FEE = 4.00
DELIVERY_FREE_THRESHOLD = 35.00
EXTRA_CHEESE_PRICE = 1.25

order = []
# Build the same order:
# pepperoni M x2 with extra cheese
unit = MENU["pizza"]["pepperoni"]["sizes"]["M"] + EXTRA_CHEESE_PRICE
order.append({"category": "pizza", "item": "pepperoni", "size": "M", "qty": 2, "unit": unit})
# soda L x2
unit = MENU["drink"]["soda"]["sizes"]["L"]
order.append({"category": "drink", "item": "soda", "size": "L", "qty": 2, "unit": unit})
# tiramisu x1
unit = MENU["dessert"]["tiramisu"]["price"]
order.append({"category": "dessert", "item": "tiramisu", "size": None, "qty": 1, "unit": unit})

# Subtotals by category
food_sub = 0.0
drink_sub = 0.0
for li in order:
    line_total = li["qty"] * li["unit"]
    if li["category"] in ("pizza", "dessert"):
        food_sub += line_total
    else:
        drink_sub += line_total

# Discounts
has_pizza = any(li["category"] == "pizza" for li in order)
has_drink = any(li["category"] == "drink" for li in order)
combo_discount = 2.0 if (has_pizza and has_drink) else 0.0

coupon_code = "WELCOME10"
subtotal_before = food_sub + drink_sub
coupon_discount = round(0.10 * subtotal_before, 2) if coupon_code.upper() == "WELCOME10" else 0.0
total_discount = round(combo_discount + coupon_discount, 2)

# Allocate discounts proportionally between food/drink
grand = food_sub + drink_sub
if grand == 0:
    food_after = 0.0
    drink_after = 0.0
else:
    food_share = food_sub / grand
    drink_share = 1 - food_share
    food_after = round(food_sub - total_discount * food_share, 2)
    drink_after = round(drink_sub - total_discount * drink_share, 2)

subtotal_after = round(food_after + drink_after, 2)

# Delivery, tip, tax
delivery = 0.0 if subtotal_after >= DELIVERY_FREE_THRESHOLD else DELIVERY_FEE
tip_rate = 0.18
tip = round(subtotal_after * tip_rate, 2)
food_tax = round(food_after * FOOD_TAX, 2)
drink_tax = round(drink_after * DRINK_TAX, 2)
tax_total = round(food_tax + drink_tax, 2)

grand_total = round(subtotal_after + tip + tax_total + delivery, 2)

# Receipt
def money(x): return f"${x:,.2f}"

print("=" * 40)
print(f"{'Pizza Shop Receipt':^40}")
print(f"{'Microsoft':^40}")
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M'):^40}")
print("-" * 40)
for li in order:
    left = f"{li['qty']} x {li['item'].title()}{f' ({li['size']})' if li['size'] else ''}"
    right = money(li["qty"] * li["unit"])
    print(f"{left:<28}{right:>12}")
print("-" * 40)
print(f"{'Food Subtotal':<28}{money(food_sub):>12}")
print(f"{'Drink Subtotal':<28}{money(drink_sub):>12}")
if combo_discount:  print(f"{'Combo Discount':<28}-{money(combo_discount):>12}")
if coupon_discount: print(f"{'Coupon (WELCOME10)':<28}-{money(coupon_discount):>12}")
print(f"{'Subtotal (after disc.)':<28}{money(subtotal_after):>12}")
print(f"{'Tip':<28}{money(tip):>12}")
print(f"{'Tax (food+drink)':<28}{money(tax_total):>12}")
if delivery: print(f"{'Delivery':<28}{money(delivery):>12}")
print("=" * 40)
print(f"{'TOTAL':<28}{money(grand_total):>12}")
print("=" * 40)


           Pizza Shop Receipt           
               Microsoft                
            2025-10-21 22:01            
----------------------------------------
2 x Pepperoni (M)                 $27.50
2 x Soda (L)                       $6.00
1 x Tiramisu                       $5.00
----------------------------------------
Food Subtotal                     $32.50
Drink Subtotal                     $6.00
Combo Discount              -       $2.00
Coupon (WELCOME10)          -       $3.85
Subtotal (after disc.)            $32.65
Tip                                $5.88
Tax (food+drink)                   $2.71
Delivery                           $4.00
TOTAL                             $45.24


### Project structure of version 2

```bash
your_project/
  utils.py     # utility functions
  main.py      # main file
```

In [ ]:
# version 2

from datetime import datetime
from utils import add_item, print_receipt

MENU = {
    "pizza": {
        "margherita": {"sizes": {"S": 8.50, "M": 11.00, "L": 13.50}},
        "pepperoni":  {"sizes": {"S": 9.50, "M": 12.50, "L": 15.00}},
    },
    "drink": {
        "soda": {"sizes": {"S": 2.00, "M": 2.50, "L": 3.00}},
        "tea":  {"sizes": {"S": 2.00, "M": 2.50, "L": 3.00}},
    },
    "dessert": {
        "tiramisu": {"price": 5.00},
    },
}

FOOD_TAX = 0.08     # 8% on food (pizza, dessert)
DRINK_TAX = 0.10    # 10% on drinks
DELIVERY_FEE = 4.00
DELIVERY_FREE_THRESHOLD = 35.00
EXTRA_CHEESE_PRICE = 1.25  # per pizza

if __name__ == "__main__":
    order = []
    add_item(order, category="pizza",  item="pepperoni", size="M", qty=2, extra_cheese=True)
    add_item(order, category="drink",  item="soda", size="L", qty=2)
    add_item(order, category="dessert",item="tiramisu", qty=1)

    print_receipt(order, customer="Microsoft", tip_rate=0.18, coupon_code="WELCOME10")


# Topics to be covered
- Functions
- Recursion
- Comprehensions
- Set

# Functions

DRY - Don't repeat yourself

If you repeat the same logic in program multiple times:
- You have to fix bugs in multiple places.
- It’s easy to make inconsistent updates.
- The code gets longer, harder to maintain, and error-prone.

In [3]:
# def tells python we are making a function, (x) says it takes one parameter
# return tells python to replace the function call with this value
def add_two(x):
    return x + 2

In [4]:
# calling the function returns the value defined in the return
add_two(2)


4

### Positional argument 
A positional argument is an argument that is passed to a function based on its position or order. They are assigned to the function parameters in the order in which they appear.


- **Order matters**: The first argument passed corresponds to the first parameter in the function definition, the second argument to the second parameter, and so on.
- **Fixed structure**: When calling a function, you must provide the arguments in the same order as defined in the function signature.
- **Mandatory**: If a positional argument is required (i.e., the function doesn't have default values), you must pass it when calling the function.

In [6]:
def add(x, y):
    print(f"x is {x} and y is {y}")
    return x + y  # Return values with a return statement

# Calling functions with parameters
print(add(5, 6))  # => prints out "x is 5 and y is 6" and returns 11

x is 5 and y is 6
11


In [ ]:
print(add(6, 5))  # => prints out "x is 5 and y is 6" and returns 11

In [ ]:
print(add(5))  # => prints out "x is 5 and y is 6" and returns 11

### Keyword argument
A keyword argument is an argument passed to a function by explicitly **specifying the name of the parameter along with its value**, rather than just relying on the position of the argument in the function call. This allows you to provide arguments **in any order**, as long as the parameter names are specified.

- **Order doesn't matter**: Unlike positional arguments, keyword arguments can be provided in any order because you explicitly specify the name of the parameter.
- **More readable**: Using keyword arguments can make function calls clearer, especially when dealing with many parameters or optional parameters.
- **Optional**: Many functions use keyword arguments for parameters that have default values, so you can choose to override them if needed.
- **Named assignment**: You explicitly assign values to specific parameters by name in the function call.

In [ ]:
# Another way to call functions is with keyword arguments
print(add(y=6, x=5))  # Keyword arguments can arrive in any order.

In [ ]:
def my_func(x, y=0):
    return x, y
my_func(1)
my_func(1,2)

def create_player(name, score=0):
    return {"name": name, "score": score}

print(create_player("Ian", 9999))

def some_func(x, y, is_debug=False):
    if is_debug:
        print(f"x: {x}, y: {y}")
    return x+y

x = input("what is x")
y = input("what is y")
some_func(x, y)

(1, 2)

### Type Hints
You can annotate function parameters (and the return value) with type hints. 

They don’t enforce types at runtime by default, but they make code easier to read and let tools like mypy or pyright catch bugs before you run the code.

In [ ]:
# example of type hints
def add(a: int, b: int) -> int:
    return a + b

## Practice Exercise with Functions 1

In [ ]:
# write a function that sums the items in a list
"""
sum_list([1,2,3])
-> 6
"""

def sum_list(my_list):
    # add your code below
    

sum_list([1,2,3])

In [ ]:
def sum_list(my_list):
    return sum(my_list)
sum_list([1,2,3])

## Practice Exercise with Functions 2

In [ ]:
# write a function that takes a string
# as input and returns the string backwards (reversed)

# add your code below
def reverse_str(my_str):

    return my_str[::-1]

reverse_str("UW")

## Practice Exercise with Functions 3

In [ ]:
# write a function that takes 2 parameters and returns them in a list
"""
combine_elements(1,2)
-> [1,2]
"""
def combine_elements(x,y):
    return [x,y]

combine_elements(1,2)

# list()

In [ ]:
# call the function sum_list with your new function as the parameter
print(sum_list(combine_elements(1,4)))

# Practice 4: Normalization
Complete the function in the cell below. The input argument is a list of numbers. The function returns a normalized list. Assume the sum of numbers in the list is non-zero.

In [ ]:
# add your code below
def normalize(l):

   

# test your code using the examples below
l1 = [1, 2, 3]
normalized_l1 = normalize(l1)
print(normalized_l1)

l2 = list(range(20))
normalized_l2 = normalize(l2)
print(normalized_l2)


In [ ]:
# solve the normalization problem using comprehension
def normalize(l):
    
# test your code using the examples below
l1 = [1, 2, 3]
normalized_l1 = normalize(l1)
print(normalized_l1)

# Practice: Get price

Write a function `get_unit_price(category: str, item: str, size: str | None = None) -> float` for your pizza shop

In [ ]:
# add your code below

## Optional Advanced Function topics
In some situations, we do not know the number of positional arguments in advance. We need to let our functions accept **a variable number of positional arguments**.

*args tells Python to take a variable number of positional arguments. These arguments are passed to a tuple named args.

In [ ]:
# You can define functions that take a variable number of
# positional arguments
def varargs(*args):
    return args

varargs(1, 2, 3)  # => (1, 2, 3)

In [10]:
# Example: sum function that takes variable number of arguments
def sum(*args):
    total = 0
    for x in args:
        total += x
    return total

sum(1,2,3)
sum(1,2,3,4,5,6,7,8,9)

45

**kwargs tells Python to take a variable number of keyword arguments. These arguments are passed to a dictionary named kwargs.

In [ ]:
# You can define functions that take a variable number of
# keyword arguments, as well
def keyword_args(**kwargs):
    return kwargs

# Let's call it to see what happens
keyword_args(big="foot", loch="ness")  # => {"big": "foot", "loch": "ness"}

In [11]:
# You can do both at once, if you like
def all_the_args(*args, **kwargs):
    print(args)
    print(kwargs)

all_the_args(1, 2, a=3, b=4)

(1, 2)
{'a': 3, 'b': 4}


In [ ]:
# When calling functions, you can do the opposite of args/kwargs!
# Use * to expand tuples and use ** to expand kwargs.
args = (1, 2, 3, 4)
kwargs = {"a": 3, "b": 4}
all_the_args(*args)            # equivalent: all_the_args(1, 2, 3, 4)
all_the_args(**kwargs)         # equivalent: all_the_args(a=3, b=4)
all_the_args(*args, **kwargs)  # equivalent: all_the_args(1, 2, 3, 4, a=3, b=4)

# Recursion
<img src = "recursion.png" width = "1000">

image credit: https://craftofcoding.wordpress.com/2022/02/17/recursion-and-problem-solving/

```
def count_doll(nested_doll): 
        return 1 + number of dolls inside it
```
We can again count the number of dolls inside by calling the `count_doll` function, with a different argument.

This process can continue until we reach the base case, where the doll cannnot be opened further.

In [ ]:
# Countdown function: Develop a function that counts down to zero from a starting number

# calling a function that calls itself is called recursion
# every recursive function needs a 'base' case to know when to stop

# what is the 'base' case here, what would happen without it?
# the 'recursive case' is a smaller version of the problem, what is it here?

def count_down(start):
    """ Count down from a number  """
    print(start)
    if start > 1:
      count_down(start-1)

count_down(3)

In [ ]:
# Example: calculate the sum of a sequence iteratively
# assume n is non-negative

def sum(n):
    total = 0
    for index in range(n+1):
        total += index
    return total

result = sum(100)
print(result)

We can also sovle the problem recursively

In [ ]:
# Example: calculate the sum of a sequence recursively
# Note, we are not printing the sum but passing the answer back
# through the recursive calls by 'building' it

def sum(n):
    if n > 0:
        return n + sum(n-1)
    return 0

result = sum(100)
print(result)

In [ ]:
# the pythonic the ternary operator(if-else in a single line), the sum() will be even more concise
def sum(n):
    return n + sum(n-1) if n > 0 else 0

result = sum(100)
print(result)

# more on ternary operator
num = 4
is_even = False if num%2 else True
print(is_even)

## Practice Exercise on Recursive Functions

In [ ]:
# write a recursive function to sum a list of numbers

def recursive_list_sum(num_list):
  # base case
  if len(num_list) == 1:
    # your code
  # recursive case
  else:
    # your code

print(recursive_list_sum([1,9,5,5]))

# Comprehensions

Every Pythonista should know this

Writing Comprehensions is very Pythonic

In [ ]:
"""
upper_case(['apple', 'orange'])
-> ['APPLE', 'ORANGE']

hint: "apple".upper() => "APPLE"
      [].append("APPLE") => ["APPLE"]
      ["APPLE"].append("ORANGE") => ["APPLE", "ORANGE"]
"""
def upper_case(my_list):
    upper_list = []
    for fruit in my_list:
        upper_fruit = fruit.upper() # APPLE
        upper_list.append(upper_fruit)
    return upper_list
upper_case(['apple', 'orange'])

def upper_case(my_list):
    for idx, fruit in enumerate(my_list):
        upper_fruit = fruit.upper() # APPLE
        my_list[idx] = upper_fruit
    return my_list
upper_case(['apple', 'orange'])

In [ ]:
my_list = ["apple", "orange"]
[fruit.upper() for fruit in my_list]

In [ ]:
"""
return list of fruits that starts with 'a',
and make it upper case
upper_a_fruits(['apple', 'orange'])
-> ['APPLE']
"""
def upper_a_fruits(my_list):
    upper_list = []
    for fruit in my_list:
        if not fruit.startswith('a'):
            continue
        upper_fruit = fruit.upper() # APPLE
        upper_list.append(upper_fruit)
    return upper_list
upper_a_fruits(['apple', 'orange'])

In [ ]:
my_list = ["apple", "orange", "banana"]
[fruit.upper() for fruit in my_list if fruit.startswith('a')]
# [transform <for-loop> filter]

In [ ]:
# Change string to all cap using .upper()
print("dog".upper())

animals = ["dog", "cat", "mouse"]
print(animals)

# take one animal and make it all cap
animals[0].upper()
print(animals[0])

# what if we want to make a list of animals all capitalized?
capitalized_animals = []
for animal in animals:
    cap_animal = animal.upper()
    capitalized_animals.append(cap_animal)
capitalized_animals

# a pythonic way: "comprehensions" creates new sequences from existing ones
# applies to lists, dictionaries, sets, generators
# []'s define a list, a.upper() describes what to do, for statement selects what
[a.upper() for a in animals]

In [ ]:
# comprehensions can be used for filtering
print([a for a in animals if a == 'dog'])

# filtering and also transforming
print([a.upper() for a in animals if a == 'dog' or a == 'mouse'])

# real world example of list of dictionaries
employees = [
    {"id": 1, "name": "Ian", "score": 5, "is_active": True},
    {"id": 2, "name": "Luyao", "score": 6, "is_active": False},
]
high_score_employees = [x for x in employees if x['score'] > 5]
print(high_score_employees)

active_employees = [x for x in employees if x['is_active']]
print(active_employees)

## Practice Exercise Comprehension 1

In [ ]:
# create a new list by adding 6 to the elements of an existing list
prev_list = [1,5,2,56]


## Optional Advanced Practice Exercise Comprehension 2

In [ ]:
# You have 100 elements in a list, but a remote storage system can only
# accept 10 elements at a time. Write a program that feeds the data to a
# remote system 10 elements at a time

# define a list of 100 elements using list functions
elements = # your code

# use list slicing and range function in a list comprehension to create
# a new list of 10-element lists
batches_of_10 =  # your code

# print (send) one batch at a time to the remote storage system
for next_ten in batches_of_10:
  print(next_ten)

## Optional Advanced Function Topics

In [ ]:
# Returning multiple values (with tuple assignments)

def swap(x, y):
    return y, x  # Return multiple values as a tuple without the parenthesis.
                 # (Note: parenthesis have been excluded but can be included)

x = 1
y = 2
x, y = swap(x, y)     # => x = 2, y = 1
print(x, y)

(x, y) = swap(x,y)  # Again the use of parenthesis is optional.
print(x, y)

In [ ]:
# global scope
x = 5

def set_x(num):
    # local scope begins here
    # local var x not the same as global var x
    x = num    # => 43
    print(x)   # => 43

def set_global_x(num):
    # global indicates that particular var lives in the global scope
    global x
    print(x)   # => 5
    x = num    # global var x is now set to 6
    print(x)   # => 6

set_x(43)
set_global_x(6)

### Docstring
Documentation string (docstring) can be used to describe the usage of functions, classes, and modules. It is good to have a docstring associated with your function to improve code readability.

In [12]:
def swap(x, y):
    """
    Swaps two numbers and return them
    
    Args:
    x: first number
    y: second number

    Returns:
    y and x in this order
    """
    return y, x 


help(swap)
                

Help on function swap in module __main__:

swap(x, y)
    Swaps two numbers and return them

    Args:
    x: first number
    y: second number

    Returns:
    y and x in this order



# Set

## What is a Set?
A set is a data structure in Python that is used to store a collection of unique elements. Unlike lists or tuples, sets do not allow duplicate values. Sets are often used to perform operations like union, intersection, and difference on data.

## Creating a Set
In Python, you can create a set by enclosing a comma-separated sequence of elements within curly braces `{}` or by using the `set()` constructor. Here's how you can create a set:

In [ ]:
# Creating a set using curly braces
my_set = {1, 2, 3, 4, 5}

# Use the set() constructor to convert a list into a set
another_set = set([2, 4, 6, 8])

## Basic Set Operations
### Adding Elements

You can add elements to a set using the `add()` method. The element should be unique; duplicate elements won't be added.

In [ ]:
my_set = {1, 2, 3, 4, 5}

my_set.add(6)
my_set

### Removing Elements
You can remove elements from a set using the `remove()` method. If the element is not present in the set, it will raise a KeyError. To avoid this, you can use the `discard()` method, which won't raise an error if the element is not found.

In [ ]:
my_set = {1, 2, 3, 4, 5}

my_set.remove(3)
my_set.discard(7)  # No error even if 7 is not in the set
my_set

In [ ]:
# can be for loop'ed just like lists
for ele in my_set:
    print(ele)

{x for x in my_set if x > 2} # set comprehension

## Set Operations

Sets support various set operations such as union, intersection, and difference.

**Union**: Combines two sets and returns a new set containing all unique elements from both sets.

In [ ]:
set1 = {1, 2, 3}
set2 = {3, 4, 5}
union_set = set1.union(set2)
union_set

**Intersection**: Returns a new set containing common elements between two sets.

In [ ]:
set1 = {1, 2, 3}
set2 = {3, 4, 5}
intersection_set = set1.intersection(set2)
intersection_set

**Difference**: Returns a new set containing elements that are in the first set but not in the second set.

In [ ]:
set1 = {1, 2, 3}
set2 = {3, 4, 5}
difference_set = set1.difference(set2)
difference_set

## Use Cases
Sets are handy in various scenarios. Here are a few use cases:

### Removing Duplicates

If you have a list of elements with duplicates, you can convert it into a set to remove the duplicates easily.

In [ ]:
original_list = [1, 2, 2, 3, 4, 4, 5]
unique_set = set(original_list)
union_set

### Finding Common Elements

Sets can be used to find common elements between two lists.

In [ ]:
list1 = [1, 2, 3, 4, 5]
list2 = [4, 5, 6, 7, 8]
common_elements = set(list1).intersection(set(list2))
common_elements

## Practice on Set Operation
1. Write a Python function that takes two sets as input and returns their union.
1. Create a set of your favorite movies. Then, write a program that asks the user to input a movie and checks if it's in your set of favorite movies.
1. Given two sets representing the courses you are taking this semester and the courses your friend is taking, write a program to find the courses you both have in common (intersection).

# Debugging Challenge

In [ ]:
# given an even integer, return sum of even numbers [0,number] (inclusive)
# given an odd integer, return sum of odd numbers [0,number] (inclusive)
number = int(input("Enter an integer: "))

i = 1

while i < nomber:
    if number % 2 = 0:
        total = total + number
    i = i + 1

print(total)